# Hyperparameter Tuning using HyperDrive
TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [32]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core import Environment, ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil

# Dataset
TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

## Data set is taken from https://archive.ics.uci.edu/ml/datasets/bank+marketing
## Source:

[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014


## Data Set Information:

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

There are four datasets:
1) bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]
2) bank-additional.csv with 10% of the examples (4119), randomly selected from 1), and 20 inputs.
3) bank-full.csv with all examples and 17 inputs, ordered by date (older version of this dataset with less inputs).
4) bank.csv with 10% of the examples and 17 inputs, randomly selected from 3 (older version of this dataset with less inputs).
The smallest datasets are provided to test more computationally demanding machine learning algorithms (e.g., SVM).

The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).


Attribute Information:

Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')


In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '3d1a56d2-7c81-4118-9790-f85d1acf0c77'
resource_group = 'aml-quickstarts-196654'
workspace_name = 'quick-starts-ws-196654'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Bankdata')
dataset.to_pandas_dataframe()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AZNXST39R to authenticate.


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [4]:
#Dataser was uploaded external from this link to ws

from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="udacity-AzureML1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-196654
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-196654


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'cluster'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS12_v2", max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found the compute cluster


# Hyperdrive Configuration
TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import shutil
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        "--C" :        choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        "--max_iter" : choice(50,100,200,300)
    }
)
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# Specify a Policy
#Your Code Here
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
 ### YOUR CODE HERE ###
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)
myenv = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#est = ScriptRunConfig(
   # source_directory= script_folder,
    #compute_target=compute_target,
    #entry_script="train.py",
    #environment=myenv
#)
est = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=myenv)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    run_config=est,
    hyperparameter_sampling = ps, 
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 15,
    max_concurrent_runs = 3,
    policy = policy
    #estimator = est
)

# Run Details
OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the RunDetails widget to show the different experiment

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)


In [10]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
hyperdrive_run.wait_for_completion(show_output = True)

RunId: HD_60f5edaa-714a-4688-a18f-64edeec25df4
Web View: https://ml.azure.com/runs/HD_60f5edaa-714a-4688-a18f-64edeec25df4?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-196654/workspaces/quick-starts-ws-196654&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-05-25T07:18:13.324030][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n"<START>[2022-05-25T07:18:14.1159098Z][SCHEDULER][INFO]Scheduling job, id='HD_60f5edaa-714a-4688-a18f-64edeec25df4_0'<END><START>[2022-05-25T07:18:14.2693447Z][SCHEDULER][INFO]Scheduling job, id='HD_60f5edaa-714a-4688-a18f-64edeec25df4_1'<END>"<START>[2022-05-25T07:18:14.274393][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2022-05-25T07:18:14.3912128Z][SCHEDULER][INFO]Scheduling job, id='HD_60f5edaa-714a-4688-a18f-64edeec25df4_2'<END>"<START>[2022-05-25T07:18:12.511257][API][IN

{'runId': 'HD_60f5edaa-714a-4688-a18f-64edeec25df4',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-05-25T07:18:12.273646Z',
 'endTimeUtc': '2022-05-25T07:37:49.557613Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'b5a90f93-9454-4e70-9199-569d390469a4',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1077-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.41.0',
  'space_size': '44',
  'score': '0.9177541729893779',
  'best_child_run_id': 'HD_60f5edaa-714a-4688-a18f-64edeec25df4_9',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_60f5edaa-714a-4688-a18f-64edeec25df4_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg196654.blob.core.windows.net/azureml/Ex

# Best Model
TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:', best_run_metrics['Regularization Strength:'])
print('\n max_iter:', best_run_metrics['Max iterations:'])

Best Run Id:  HD_60f5edaa-714a-4688-a18f-64edeec25df4_9

 Accuracy: 0.9177541729893779

 C: 0.1

 max_iter: 300


In [13]:
best_run.get_details()['runDefinition']['arguments']


['--C', '0.1', '--max_iter', '300']

In [14]:
best_run.get_metrics(name='Accuracy')

{'Accuracy': 0.9177541729893779}

In [15]:
print(best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_4fc4c501f8d42aba63419235e93fb3be1b852270c9a07fec58e362ef609133f3_d.txt', 'azureml-logs/65_job_prep-tvmps_4fc4c501f8d42aba63419235e93fb3be1b852270c9a07fec58e362ef609133f3_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_4fc4c501f8d42aba63419235e93fb3be1b852270c9a07fec58e362ef609133f3_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/96_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [16]:
print(best_run)

Run(Experiment: udacity-AzureML1,
Id: HD_60f5edaa-714a-4688-a18f-64edeec25df4_9,
Type: azureml.scriptrun,
Status: Completed)


# Model Deployment
Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
model = best_run.register_model('BankMarketingModel', model_path='outputs/model.joblib')

In [18]:
best_run.download_file('outputs/model.joblib', 'models/model.joblib')

In [19]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-196654', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-196654'), name=BankMarketingModel, id=BankMarketingModel:1, version=1, tags={}, properties={})

In [20]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice

In [21]:
#myenv = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

inference_config = InferenceConfig(entry_script="score.py", environment = myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1,enable_app_insights = True)

In [22]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

inference_config = InferenceConfig(entry_script="score.py", environment = myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1,enable_app_insights = True)

In [23]:

from azureml.core.model import Model
service=Model.deploy(workspace=ws,
                    name="bank-service-final",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-05-25 07:38:48+00:00 Creating Container Registry if not exists.
2022-05-25 07:38:48+00:00 Registering the environment.
2022-05-25 07:38:49+00:00 Use the existing image.
2022-05-25 07:38:49+00:00 Generating deployment configuration.
2022-05-25 07:38:50+00:00 Submitting deployment to compute.
2022-05-25 07:38:53+00:00 Checking the status of deployment bank-service-final..
2022-05-25 07:40:51+00:00 Checking the status of inference endpoint bank-service-final.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [24]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://9fad921a-8026-4d41-add5-96e42c5c937f.southcentralus.azurecontainer.io/score

swagger URI: 
http://9fad921a-8026-4d41-add5-96e42c5c937f.southcentralus.azurecontainer.io/swagger.json

http://9fad921a-8026-4d41-add5-96e42c5c937f.southcentralus.azurecontainer.io/score
http://9fad921a-8026-4d41-add5-96e42c5c937f.southcentralus.azurecontainer.io/swagger.json


In [25]:
import requests
import json
data={"data":
[{"age": 17, 
"campaign": 1,
"cons.conf.idx": -46.2, 
"cons.price.idx": 92.893, 
"contact": "cellular", 
"day_of_week": "mon", 
"default": "no", 
"duration": 971, 
"education": "university.degree", 
"emp.var.rate": -1.8, 
"euribor3m": 1.299, 
"housing": "yes", "job": 
"blue-collar", "loan": "yes", 
"marital": "married", 
"month": "may", 
"nr.employed": 5099.1, 
"pdays": 999, 
"poutcome": "failure", 
"previous": 1}]
}
input_data=json.dumps(data)
with open("data.json","w") as file:
    file.write(input_data)
headers={"Content-Type":"application/json"}
result=requests.post(scoring_uri, input_data, headers=headers)


In [26]:
result.status_code

200